# Capítulo 7: Construindo Aplicações de Chat
## Introdução Rápida à API Azure OpenAI


## Visão Geral
Este notebook é adaptado do [Repositório de Exemplos do Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), que inclui notebooks que também acessam o serviço [OpenAI](notebook-openai.ipynb).

A API Python OpenAI funciona com o Azure OpenAI também, com algumas modificações. Saiba mais sobre as diferenças aqui: [Como alternar entre os endpoints OpenAI e Azure OpenAI com Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

Para mais exemplos de início rápido, consulte a documentação oficial [Azure OpenAI Quickstart](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst)


## Índice  

[Visão geral](../../../../07-building-chat-applications/python)  
[Primeiros passos com o Azure OpenAI Service](../../../../07-building-chat-applications/python)  
[Crie seu primeiro prompt](../../../../07-building-chat-applications/python)  

[Casos de uso](../../../../07-building-chat-applications/python)    
[1. Resumir texto](../../../../07-building-chat-applications/python)  
[2. Classificar texto](../../../../07-building-chat-applications/python)  
[3. Gerar novos nomes de produtos](../../../../07-building-chat-applications/python)  
[4. Ajustar um classificador](../../../../07-building-chat-applications/python)  
[5. Embeddings](../../../../07-building-chat-applications/python)

[Referências](../../../../07-building-chat-applications/python)


### Começando com o Azure OpenAI Service

Novos clientes precisam [solicitar acesso](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst) ao Azure OpenAI Service.  
Depois que o acesso for aprovado, os clientes podem entrar no portal do Azure, criar um recurso do Azure OpenAI Service e começar a testar os modelos pelo estúdio  

[Ótimo recurso para começar rapidamente](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### Crie seu primeiro prompt  
Este exercício rápido vai te dar uma introdução básica sobre como enviar prompts para um modelo da OpenAI para uma tarefa simples de "resumo".


**Passos**:  
1. Instale a biblioteca OpenAI no seu ambiente Python  
2. Carregue as bibliotecas auxiliares padrão e defina suas credenciais de segurança típicas da OpenAI para o serviço OpenAI que você criou  
3. Escolha um modelo para sua tarefa  
4. Crie um prompt simples para o modelo  
5. Envie sua solicitação para a API do modelo!


> [!NOTE] Esta etapa não é necessária se você executar este notebook no Codespaces ou dentro de um Devcontainer


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. Encontrando o modelo certo  
Os modelos GPT-3.5-turbo ou GPT-4 conseguem entender e gerar linguagem natural. O serviço oferece quatro opções de modelos, cada uma com diferentes níveis de desempenho e velocidade, ideais para diferentes tipos de tarefas.

[Modelos do Azure OpenAI](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. Design de Prompt  

“A mágica dos grandes modelos de linguagem é que, ao serem treinados para minimizar esse erro de previsão em enormes quantidades de texto, eles acabam aprendendo conceitos úteis para essas previsões. Por exemplo, eles aprendem conceitos como”(1):

* como soletrar
* como funciona a gramática
* como parafrasear
* como responder perguntas
* como manter uma conversa
* como escrever em vários idiomas
* como programar
* etc.

#### Como controlar um grande modelo de linguagem  
“De todos os inputs para um grande modelo de linguagem, de longe o mais influente é o prompt de texto”(1).

Grandes modelos de linguagem podem ser estimulados a produzir respostas de algumas formas:

- Instrução: Diga ao modelo o que você quer
- Completação: Induza o modelo a completar o início do que você deseja
- Demonstração: Mostre ao modelo o que você quer, com:
- Alguns exemplos no prompt
- Centenas ou milhares de exemplos em um conjunto de dados de treinamento para ajuste fino



#### Existem três diretrizes básicas para criar prompts:

**Mostre e explique**. Deixe claro o que você quer, seja por instruções, exemplos ou uma combinação dos dois. Se você quer que o modelo organize uma lista de itens em ordem alfabética ou classifique um parágrafo pelo sentimento, mostre que é isso que você deseja.

**Forneça dados de qualidade**. Se você está tentando construir um classificador ou fazer o modelo seguir um padrão, certifique-se de que há exemplos suficientes. Não esqueça de revisar seus exemplos — o modelo geralmente é esperto o bastante para ignorar erros básicos de ortografia e te dar uma resposta, mas também pode assumir que isso foi intencional e isso pode afetar o resultado.

**Verifique suas configurações.** As configurações de temperatura e top_p controlam o quão determinístico o modelo é ao gerar uma resposta. Se você quer uma resposta onde só existe uma resposta correta, é melhor deixar esses valores mais baixos. Se você busca respostas mais variadas, pode aumentar esses valores. O erro mais comum que as pessoas cometem com essas configurações é achar que elas controlam a “inteligência” ou “criatividade” do modelo.


Fonte: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Resumir Texto  
#### Desafio  
Resuma um texto adicionando um 'tl;dr:' ao final de um trecho. Observe como o modelo entende como realizar várias tarefas sem instruções adicionais. Você pode experimentar prompts mais descritivos do que tl;dr para modificar o comportamento do modelo e personalizar o resumo que recebe(3).  

Trabalhos recentes demonstraram avanços significativos em várias tarefas e benchmarks de PLN ao treinar previamente em um grande corpus de texto e, em seguida, ajustar para uma tarefa específica. Embora normalmente a arquitetura seja independente da tarefa, esse método ainda exige conjuntos de dados específicos para ajuste fino, com milhares ou dezenas de milhares de exemplos. Em contraste, humanos geralmente conseguem realizar uma nova tarefa de linguagem com apenas alguns exemplos ou instruções simples – algo que os sistemas atuais de PLN ainda têm dificuldade em fazer. Aqui mostramos que aumentar a escala dos modelos de linguagem melhora bastante o desempenho em tarefas gerais com poucos exemplos, às vezes até alcançando competitividade com abordagens anteriores de ajuste fino de última geração. 



Tl;dr


# Exercícios para vários casos de uso  
1. Resumir Texto  
2. Classificar Texto  
3. Gerar Novos Nomes de Produtos  
4. Embeddings  
5. Ajustar um classificador


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Classificar Texto  
#### Desafio  
Classifique itens em categorias fornecidas no momento da inferência. No exemplo a seguir, fornecemos tanto as categorias quanto o texto a ser classificado no prompt (*playground_reference).

Consulta do cliente: Olá, uma das teclas do teclado do meu notebook quebrou recentemente e vou precisar de uma reposição:

Categoria classificada:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Gerar Novos Nomes de Produtos
#### Desafio
Crie nomes de produtos a partir de palavras de exemplo. Aqui, incluímos no prompt informações sobre o produto para o qual vamos gerar nomes. Também fornecemos um exemplo semelhante para mostrar o padrão que desejamos receber. Definimos o valor de temperatura alto para aumentar a aleatoriedade e obter respostas mais inovadoras.

Descrição do produto: Um aparelho caseiro para fazer milkshake
Palavras-chave: rápido, saudável, compacto.
Nomes de produtos: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Descrição do produto: Um par de sapatos que serve em qualquer tamanho de pé.
Palavras-chave: adaptável, ajuste, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## Embeddings
Esta seção vai mostrar como obter embeddings e encontrar similaridades entre palavras, frases e documentos. Para executar os notebooks a seguir, você precisa implantar um modelo que use `text-embedding-ada-002` como modelo base e definir o nome dessa implantação no arquivo .env, usando a variável `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT`.


### Taxonomia de Modelos - Escolhendo um modelo de embedding

**Taxonomia de modelos**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  (família de embeddings)  
{capability} --> ada             (todos os outros modelos de embedding serão descontinuados em 2024)  
{input-type} --> n/a             (apenas especificado para modelos de busca)  
{identifier} --> 002             (versão 002)  

model = 'text-embedding-ada-002'


> [!NOTE] A etapa a seguir não é necessária se você executar este notebook no Codespaces ou dentro de um Devcontainer


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## Comparando artigo do conjunto de dados cnn daily news
fonte: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# Referências  
- [Documentação do Azure - Modelos do Azure OpenAI](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [Exemplos do OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)


# Para Mais Ajuda  
[Equipe de Comercialização da OpenAI](AzureOpenAITeam@microsoft.com)


# Colaboradores
* Louis Li



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional feita por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
